## Setup RPC & deployer

In [ ]:
from dotenv import load_dotenv
import os
import boa
from eth_account import Account
from web3 import Web3
import logging
import subprocess
import time

load_dotenv()
chain_handle = "base-sepolia"

PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")
RPCS = {
    "sepolia": "https://sepolia.drpc.org",
    "base-sepolia": "https://sepolia.base.org",
    "optimism-sepolia": "https://sepolia.optimism.io",
    "arbitrum-sepolia": "https://sepolia-rollup.arbitrum.io/rpc",
}

RPC_URL = RPCS[chain_handle]

deployer = Account.from_key(PRIVATE_KEY)
eth_env = boa.set_network_env(RPC_URL)
# boa.set_env(eth_env)
w3 = Web3(Web3.HTTPProvider(RPC_URL))

# this automatically sets the eoa as the deployer
boa.env.add_account(deployer)
print(f"Deploying with {deployer.address} on Chain id {boa.env.evm.patch.chain_id}")
print(f"Chain balance is {w3.eth.get_balance(deployer.address)/1e18 :.3f} ETH")

## Setup LZ

In [ ]:
from LZDeployments import LZDeployments
import json

lz = LZDeployments()
chain_data = lz.get_chain_metadata(chain_handle)
# print(json.dumps(chain_data, indent=2))
LZ_ENDPOINT = chain_data["metadata"]["endpointV2"]
LZ_EID = chain_data["metadata"]["eid"]
SEND_LIB = chain_data["metadata"].get("sendUln302", "unavailable")
RECEIVE_LIB = chain_data["metadata"].get("receiveUln302", "unavailable")
READ_LIB = chain_data["metadata"].get("readLib1002", "unavailable")
print(f"Operating on: {chain_handle}")
print(f"Chain eID: {LZ_EID}\nEndpoint address: {LZ_ENDPOINT}")
dvns_all = chain_data["dvns"]
dvns_lzread = chain_data["dvns_lzread"]
print(f"DVNs: {len(dvns_all)}, Read DVNs: {len(dvns_lzread)}")
print(f"Send lib: {SEND_LIB}\nReceive lib: {RECEIVE_LIB}\nRead lib: {READ_LIB}")

# I. Deployment

### 1. ExampleMessenger

In [ ]:
# II. Then deploy the DepositLimitModule
contract_deployer = boa.load_partial("../contracts/ExampleMessenger.vy")

contract = contract_deployer(
    LZ_ENDPOINT,  # endpoint on sepolia base
    500_000,  # default gas limit
)

print(f"Example Messenger deployed at {contract.address}")

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [ ]:
def get_vyper_abi():
    command = ["vyper", "../contracts/ExampleMessenger.vy", "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
# RPC endpoints

CONTRACT_ADDRESS = contract.address

ABI = get_vyper_abi()

contract_w3 = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)
account = w3.eth.account.from_key(deployer.key)


def send_tx_single(func, acc, value=0):
    tx = func.build_transaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "value": value,
        }
    )
    tx["gas"] = int(5 * w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(func, acc, value=0):
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(func, acc, value)
            success = True
        except Exception as e:
            if "replacement transaction underpriced" in str(e) or "nonce too low" in str(e):
                print(str(e), "Retrying...")
                success = False
                time.sleep(0.5)
            else:
                raise e
    return tx_hash

## Prepare delegate and config

In [ ]:
# send lib
func = contract_w3.functions.set_lz_send_lib(LZ_EID, SEND_LIB)
tx_hash1 = send_tx(func, account)
print(f"Added send lib: {tx_hash1.hex()}")
# receive lib
func = contract_w3.functions.set_lz_receive_lib(LZ_EID, RECEIVE_LIB)
tx_hash2 = send_tx(func, account)
print(f"Added receive lib: {tx_hash2.hex()}")

In [ ]:
from lzreadabi import lzreadlib_abi

lzreadlib = w3.eth.contract(address=READ_LIB, abi=json.loads(lzreadlib_abi))

print(f"Read lib type: {lzreadlib.functions.messageLibType().call()}")
print(f"Read lib version: {lzreadlib.functions.version().call()}")
eid_check = [4294967295, 4294967294]
supported_eid = None
for eid in eid_check:
    support = lzreadlib.functions.isSupportedEid(eid).call()
    print(f"Supports {eid}: {support}")
    if support and not supported_eid:
        supported_eid = eid
## read lib
read_channel = contract_w3.functions.LZ_READ_CHANNEL().call()
if supported_eid and read_channel != supported_eid:
    print(f"Setting read channel to {supported_eid}")
    func = contract_w3.functions.set_lz_read_channel(supported_eid)
    tx_hash = send_tx(func, account)
    print(f"Set read channel: {tx_hash.hex()}")

# read direction
read_channel = contract_w3.functions.LZ_READ_CHANNEL().call()
print(f"Read Channel: {read_channel}")
func = contract_w3.functions.set_lz_send_lib(read_channel, READ_LIB)
tx_hash3 = send_tx(func, account)
print(f"Added read lib: {tx_hash3.hex()}")

# receive direction
func = contract_w3.functions.set_lz_receive_lib(read_channel, READ_LIB)
tx_hash4 = send_tx(func, account)
print(f"Added receive lib: {tx_hash4.hex()}")

## Set up DVNs

In [ ]:
# first set the delegate
# func = contract_w3.functions.set_lz_delegate(account.address)
# tx_hash = send_tx(func, account)
# logging.info(f"Added self as delegate: {tx_hash.hex()}")
def checksum(address):
    return Web3.to_checksum_address(address)


CONFIG_TYPE_READ = 1
CONFIG_TYPE_ULN = 2
# list all dvns
print("Listing all dvns")
for dvn in dvns_all:
    print(dvn)

oapp = contract.address
confirmations = 1
required_dvns = [dvn["address"] for dvn in dvns_all if dvn["id"] == "layerzero-labs"]
required_dvns = required_dvns[0:1]  # of all eid dvns we pick first one with layerzero id
optional_dvns = [dvn["address"] for dvn in dvns_all if dvn["address"] not in required_dvns]
# all other dvs are optional, with a threshold of 2 (or less is there is not enough)
optional_dvn_threshold = min(2, len(optional_dvns))  # 2 or 1

# sort dvns alphabetically (important, otherwise lz will fail!)
required_dvns.sort()
optional_dvns.sort()
required_dvns = [checksum(dvn) for dvn in required_dvns]
optional_dvns = [checksum(dvn) for dvn in optional_dvns]

print("DVNs for send/receive:")
print(f"Required: {required_dvns}")
print(f"Optional: {optional_dvns}")
print(f"Optional threshold: {optional_dvn_threshold}")
for lib in [SEND_LIB, RECEIVE_LIB]:
    func = contract_w3.functions.set_lz_uln_config(
        LZ_EID,
        oapp,
        lib,
        CONFIG_TYPE_ULN,
        confirmations,
        required_dvns,
        optional_dvns,
        optional_dvn_threshold,
    )
    tx_hash = send_tx(func, account)
    print(f"Added ULN config: {tx_hash.hex()}")
    # break

required_dvns = [
    dvn["address"]
    for dvn in dvns_all
    if dvn["id"] == "layerzero-labs" and dvn.get("lzReadCompatible", False)
]
optional_dvns = [
    dvn["address"]
    for dvn in dvns_all
    if dvn["address"] not in required_dvns and dvn.get("lzReadCompatible", False)
]
optional_dvn_threshold = min(2, len(optional_dvns))  # 2 or 1 or 0
print("DVNs for read:")
print(f"Required: {required_dvns}")
print(f"Optional: {optional_dvns}")
print(f"Optional threshold: {optional_dvn_threshold}")
required_dvns.sort()
optional_dvns.sort()
required_dvns = [checksum(dvn) for dvn in required_dvns]
optional_dvns = [checksum(dvn) for dvn in optional_dvns]
# required_dvns = []
for lib in [READ_LIB]:
    func = contract_w3.functions.set_lz_uln_config(
        read_channel,
        oapp,
        lib,
        CONFIG_TYPE_READ,
        confirmations,
        required_dvns,
        optional_dvns,
        optional_dvn_threshold,
    )
    tx_hash = send_tx(func, account)
    print(f"Added ULN-Read config: {tx_hash.hex()}")

## 1. LZ Send & Receive

In [ ]:
# add self as peer

func = contract_w3.functions.set_peer(LZ_EID, CONTRACT_ADDRESS)
tx_hash = send_tx(func, account)
logging.info(f"Added self as peer tx: {tx_hash.hex()}")

In [ ]:
gas_limit = 500_000
msg = "170475436437825620930817601234267694881687829390282260281137596999800372275961"

fee = contract_w3.functions.quote_message_fee(
    LZ_EID, CONTRACT_ADDRESS, msg, _gas_limit=gas_limit
).call()
logging.info(f"Fee: {fee}")

func = contract_w3.functions.send_message(
    _dst_eid=LZ_EID, _receiver=CONTRACT_ADDRESS, _message=msg, _gas_limit=gas_limit
)
tx_hash = send_tx(func, account, 3 * fee)
logging.info(f"Tx: {tx_hash.hex()}")
# after we sent the message we can check it on testnet.layerzeroscan.com (use address or txid)
# shortly after, it should be pinged back (we send to ourselves, and event be fired)

## 2. LzRead

In [ ]:
# set self as read peer
# LZ_READ_CHANNEL = 4294967294
LZ_READ_CHANNEL = contract_w3.functions.LZ_READ_CHANNEL().call()
func = contract_w3.functions.set_peer(LZ_READ_CHANNEL, CONTRACT_ADDRESS)
tx_hash = send_tx(func, account)
logging.info(f"Set peer tx: {tx_hash.hex()}")

In [ ]:
# prepare reading calldata
from vyper.utils import method_id

method_str = "dummy_endpoint(uint256)"
num = 34710
calldata = method_id(method_str) + boa.util.abi.abi_encode("(uint256)", (num,))

# read the contract itself
res = contract_w3.functions.dummy_endpoint(num).call()

logging.info(f"Read: {res}")

# now quote lzread fee
fee = contract_w3.functions.quote_read_fee(
    LZ_EID, CONTRACT_ADDRESS, calldata, _gas_limit=gas_limit
).call()
logging.info(f"Fee: {fee}")

# now request read
func = contract_w3.functions.request_read(LZ_EID, CONTRACT_ADDRESS, calldata, _gas_limit=gas_limit)
tx_hash = send_tx(func, account, 3 * fee)
logging.info(f"Tx: {tx_hash.hex()}")

In [ ]:
# test calldata
import requests

print(calldata.hex())

rpc_endpoint = "https://sepolia.base.org"
# perform eth_call request to contract using calldata
r = requests.post(
    rpc_endpoint,
    json={
        "jsonrpc": "2.0",
        "method": "eth_call",
        "params": [{"to": CONTRACT_ADDRESS, "data": "0x" + calldata.hex()}, "latest"],
        "id": 1,
    },
)
response = r.json()["result"]
# print(r.json()['result'])
print(response)
contract_code = """
@external
@view
def message_as_string(_message: Bytes[128]) -> String[128]:
    message: String[128] = convert(_message, String[128])
    return message
"""
with boa.swap_env(boa.Env()):
    tmp_contract = boa.loads(contract_code)

    res = tmp_contract.message_as_string(bytes.fromhex(response[2:]))
    print(res)